In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
# 1. Data Preparation and Loading
# create **known** parameters
weight = 0.7
bias = 0.3

# create 
start = 0
end = 1
steps = 0.02
X = torch.arange(start, end, steps).unsqueeze(dim=1) # add 1 dims

y = weight * X + bias
X[:10]
y[:10]

tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])

In [4]:
len(X), len(y)

(50, 50)

In [5]:
## Spliting data into training and test sets    
# traing, validation and test set

# creating training set
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

In [6]:
#a# Visualize, Visualize, Visualize
import matplotlib.pyplot as plt

In [7]:
def plot_predictions(train_data=X_train, 
                     train_labels=y_train, 
                     test_data=X_test, 
                     test_labels=y_test, 
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """
  plt.figure(figsize=(10, 7))

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
  
  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14})
  plt.show()

In [8]:
# plot_predictions()

#### Build a model
Create linear regression model class
 building classes that can use the following link:
https://realpython.com/python-classes/


In [9]:
# what model does:
# start with random weights and bias
# look at the training data and adjust weights and bias to better represent the data

# through two main algorithms:
# 1. Gradient Descent
# 2. Backpropagation

In [10]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # initialize model parameters (weights and bias)
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad = True,
                                                dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad = True,
                                             dtype = torch.float))
        # Forward method to define the computation in the model
    def forward(self,x: torch.Tensor) -> torch.Tensor: # x is input data
        return self.weights * x + self.bias

In [11]:
# Through two main algorithms
# 1. Gradient descent
# 2. Backpropagation

### Pytorch model biulding essential 

__torch.nn__ - contains all of the buildings for computational graphs

__torch.nn.Parameter__ - what parameters should our model try and learn

__torch.nn.Module__ - The base class for all neural network modules

__torch.optim__ - this where the optimizers in PyTorch live, help with gradient descent

__def forward()__ - All nn.Module subclasses require you to overwrite forward()



In [12]:
model_0 = LinearRegressionModel()

## access to the model parameters
model_0.parameters()

torch.manual_seed(42)   # reproducibility of the model parameters

list(model_0.parameters())

## the better way to access model parameters
model_0.state_dict()


OrderedDict([('weights', tensor([1.0908])), ('bias', tensor([0.7446]))])

In [13]:
### ideal parameters are predefined earlier
# weight = 0.7
# bias = 0.3

# the model task is to start from random parameters and learn the ideal parameters
# making prediction using `torch.inference_mode()`


In [14]:
X_test, y_test

(tensor([[0.8000],
         [0.8200],
         [0.8400],
         [0.8600],
         [0.8800],
         [0.9000],
         [0.9200],
         [0.9400],
         [0.9600],
         [0.9800]]),
 tensor([[0.8600],
         [0.8740],
         [0.8880],
         [0.9020],
         [0.9160],
         [0.9300],
         [0.9440],
         [0.9580],
         [0.9720],
         [0.9860]]))

In [15]:
## how model predicts `y_test` based on `X_test`
with torch.inference_mode():   # this disables gradient tracking, make pytorch use less memory
    y_preds = model_0(X_test)
y_preds

## alternatively one can use
# with torch.no_grad():
#     y_preds = model_0(X_test)
# y_preds


tensor([[1.6173],
        [1.6391],
        [1.6609],
        [1.6828],
        [1.7046],
        [1.7264],
        [1.7482],
        [1.7700],
        [1.7918],
        [1.8137]])

In [15]:
## lets compare predictions with ideal values

import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))

plt.scatter(X_train, y_train, label="Training Values")
plt.scatter(X_test, y_test, label="Ideal Values", c = 'g')
plt.scatter(X_test, y_preds, label="Model Predictions", c = 'r')
plt.legend()
plt.show()    # poor model! 

: 

### Train model
training model to move from some __unknown__ parms to some __known__ parms. 

one way: use __loss function__ 

* **Loss Function** A func to measure how wrong your models pred to ideal outputs, lower is better

* __Optimizer__: takes into account the loss and adjust model's parms to improve the loss func, where we need
    * A training loop
    * A testing loop

In [16]:
list(model_0.parameters())

[Parameter containing:
 tensor([1.0908], requires_grad=True),
 Parameter containing:
 tensor([0.7446], requires_grad=True)]

In [ ]:
## kind of loss functions
## 1. Mean Absolute Error Loss --> `torch.nn.L1Loss()`          -- l_n = |y - ŷ|
## 2. Mean Squared Error Loss --> `torch.nn.MSELoss()`          -- l_n = (y - ŷ)²
## 3. Cross Entropy Loss --> `torch.nn.CrossEntropyLoss()`      -- l_n = -Σ(y * log(ŷ))

## Building a training loop in PyTorch

0. loop through the data
1. Forward pass  to make preds on data
2. Calculate the loss
3. Optomizer zero grad
4. Loss Backward 
5. Optimizer step (__gradiant descent__) 

In [18]:
## Use MAE loss
loss_fn = nn.L1Loss()

## setup an optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD(params = model_0.parameters(), lr=0.01)   # lr = learning rate

In [ ]:

torch.manual_seed(42)
# An epoch is one loop over the entire dataset
epochs = 100

### Training 
# 0. loop through the data
for epoch in range(epochs):
    # set the model to training mode
    model_0.train()   # Set the parameters to that require gradients
    # 1. Forward pass
    y_pred = model_0(X_train)

    # 2. calculate the loss
    loss = loss_fn(y_pred, y_train)     # input first, target next
    print(f'Loss: {loss.item()}')
    # 3. optimize zero gradients
    optimizer.zero_grad()

    # 4. perform backwardpropagation on the loss with respect to the model parameters
    loss.backward()

    # 5. step the optimizer (perform gradient descent)
    optimizer.step()


    model_0.eval()    # turn off gradient tracking or equivalently
    with torch.inference_mode():
        

    ## print out model state_dict
    print(model_0.state_dict())

Loss: 0.5970739722251892
OrderedDict({'weights': tensor([1.0869]), 'bias': tensor([0.7346])})
Loss: 0.585552990436554
OrderedDict({'weights': tensor([1.0830]), 'bias': tensor([0.7246])})
Loss: 0.5740319490432739
OrderedDict({'weights': tensor([1.0791]), 'bias': tensor([0.7146])})
Loss: 0.5625109672546387
OrderedDict({'weights': tensor([1.0752]), 'bias': tensor([0.7046])})
Loss: 0.5509899258613586
OrderedDict({'weights': tensor([1.0713]), 'bias': tensor([0.6946])})
Loss: 0.5394688844680786
OrderedDict({'weights': tensor([1.0674]), 'bias': tensor([0.6846])})
Loss: 0.5279479622840881
OrderedDict({'weights': tensor([1.0635]), 'bias': tensor([0.6746])})
Loss: 0.5164269208908081
OrderedDict({'weights': tensor([1.0596]), 'bias': tensor([0.6646])})
Loss: 0.5049058794975281
OrderedDict({'weights': tensor([1.0557]), 'bias': tensor([0.6546])})
Loss: 0.4933848977088928
OrderedDict({'weights': tensor([1.0518]), 'bias': tensor([0.6446])})
Loss: 0.4818638861179352
OrderedDict({'weights': tensor([1.04

In [19]:
model_0.state_dict()

OrderedDict([('weights', tensor([-0.3952])), ('bias', tensor([0.6719]))])

In [20]:
## real parameters
weight, bias

(0.7, 0.3)

In [87]:
## how model predicts `y_test` based on `X_test`

with torch.inference_mode():   # this disables gradient tracking, make pytorch use less memory
    y_preds_new = model_0(X_test)

In [21]:
## lets compare predictions with ideal values

import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))

plt.scatter(X_train, y_train, label="Training Values")
plt.scatter(X_test, y_test, label="Ideal Values", c = 'g')
plt.scatter(X_test, y_preds_new, label="Model Predictions", c = 'r')
plt.legend()
plt.show()    # poor model! 

NameError: name 'y_preds_new' is not defined

: 